# Implémentation d'un Modèle CatBoost pour l'Optimisation des Ressources d'un Système de Vélos Partagés

Ce projet vise à développer un modèle prédictif multi-cibles pour anticiper en temps réel l'offre et la demande dans les stations de vélos en libre-service (ex: Vélib'). En exploitant des données OpenData (Paris) incluant des variables temporelles, météorologiques et l'historique d'utilisation, l'objectif principal est de prédire simultanément le nombre de vélos disponibles et le nombre de bornettes libres par station.

In [68]:
import pandas as pd
import warnings
import os
import json

warnings.filterwarnings('ignore')

%matplotlib inline

In [69]:
data_folder = '../data/'

parquet_folder = data_folder + 'parquet/'
json_folder = data_folder + 'json/'
meteo_folder = data_folder + 'meteo/'

parquet_file_names = os.listdir(parquet_folder)
json_file_names = os.listdir(json_folder)
#meteo_file_names = os.listdir(meteo_folder)

print("parquet file : ", len(parquet_file_names))
print("json file : ", len(json_file_names))
#print("meteo file : ", len(json_file_names))

parquet file :  1414
json file :  110


In [70]:
parquet_data = pd.read_parquet(parquet_folder + parquet_file_names[0])
parquet_data.columns

Index(['stationcode', 'name', 'is_installed', 'capacity', 'numdocksavailable',
       'numbikesavailable', 'mechanical', 'ebike', 'is_renting',
       'is_returning', 'duedate', 'coordonnees_geo',
       'nom_arrondissement_communes', 'code_insee_commune'],
      dtype='object')

In [71]:
def load_and_format_json(json_file_path):
    with open(json_file_path) as json_file:
        data = json.load(json_file)
        return pd.DataFrame([record['fields'] for record in data['records']])

json_data = load_and_format_json(json_folder+json_file_names[0])
json_data.columns

Index(['name', 'stationcode', 'ebike', 'mechanical', 'coordonnees_geo',
       'duedate', 'numbikesavailable', 'numdocksavailable', 'capacity',
       'is_renting', 'is_installed', 'nom_arrondissement_communes',
       'is_returning', 'code_insee_commune'],
      dtype='object')

In [72]:
# Remove first element
parquet_file_names.pop(0)
json_file_names.pop(0)

'velib_2025_12_17T12_00_02_956z.json'

In [73]:
print("parquet dataframe length: ", len(parquet_data))
print("json dataframe length: ", len(json_data))

parquet dataframe length:  10
json dataframe length:  1503


In [74]:
data = pd.concat([parquet_data, json_data])
len(data)

1513

In [77]:
all_files = parquet_file_names + json_file_names
len(all_files)

1522

In [80]:
for file in all_files:
    if file.endswith('.parquet'):
        data = pd.concat([data, pd.read_parquet(parquet_folder + file)])
    if file.endswith('.json'):
        data = pd.concat([data, load_and_format_json(json_folder + file)])

len(data.shape)

534484